# Data Analysis

The purpose of this notebook is to identify the optimal data processing structure.

### Feature Selection

#### KEPT FEATURES

| Variable | Description | Type of Feature | Range |
| --- | --- | --- | --- |
| AGE | Patient age | Continuous | [1, 255] |
| AMONTH | Admission month | Categorical | [1, 12] |
| AWEEKEND | Admission day is on a weekend | Categorical | [0, 1] |
| BODYSYSTEMn | Chronic Condition Indicators - body system (2014) | Categorical | [1, 18] |
| CHRONBn | Chronic Condition Indicators - body system (≤2013) | Categorical | [1, 18] |
| CHRONn | ICD-9-CM Chronic Condition Indicators | Categorical | [0, 1] |
| CM_[comorbidity]| Presence of a comorbidity | Categorical | [0, 1] |
| DXn| ICD-9-CM Diagnosis code | Categorical | [0, 299910] |
| DXCCSn| Single-level CCS: ICD-9-CM Dx Class | Categorical | [1-259] |
| DXMCCSn | Multi-Level CCS for ICD-9-CM Dx Class | Categorical | [0, 99999999] |
| E_CCS1 | Single-level CCS for ICD-9-CM External Cause of Injury | Categorical | [2601, 2629] |
| E_MCCSn | Multi-Level CCS for ICD-9-CM External Cause of Injury	| Categorical | [0, 99999999] |
| ECODEn | ICD-9-CM External Cause of Injury Code | Categorical | [0, 9991] |
| ELECTIVE | Elective vs non-elective admission | Categorical | [0, 1] |
| FEMALE | sex/gender | Categorical | [0, 1] |
| HCUP_ED| HCUP indicator of emergency department record | Categorical | [0, 4] |
| HOSP_DIVISION | Census Division of hospital (STRATA) | Categorical | [1, 9] |
| NIS_STRATUM | Stratum used to post-stratify hospital | Categorical | [1011, 9433] |
| ORPROC | Major operating room ICD-9-CM procedure indicator | Categorical | [0, 1] |
| PAY1 | Expected primary payer | Categorical | [1, 6] |
| PCLASSn | ICD-9-CM Procedure class | Categorical | [1, 4] |
| PL_NCHSn | Category urban-rural classification scheme | Categorical | [1, 6] |
| PRn| Procedure codes |  Categorical | [0, 9999] |
| PRCCSn | CCS class for ICD-9-CM procedures | Categorical | [1, 231] |
| PRDAYn| Number of days since admission to procedure n | Continuous? | [-4, 365] |
| PRMCCS1 | Multi-Level CCS for ICD-9-CM Procedures	| Categorical | [0, 999999] |
| TRAN_IN | Indicator of transfer into the hospital | Categorical | [0, 1] |
| ZIPINC_QRTL | Mean household income for zipcode | Categorical | [1, 4] | 


#### REMOVED FEATURES

| Variable | Description | Reason for Exclusion |
| --- | --- | --- |
| AGE_NEONATE| Neonatal age | Ignore neonates for now |
| DIED | Death during hospitalization stay | OUTCOME |
| DISCWT | Weights | No useful information |
| DISPUNIFORM | disposition | OUTCOME |
| DQTR | discharge quarter | OUTCOME |
| DRG | DRG in use on discharge date | OUTCOME |
| DRG24 | DRG, Version 24 | OUTCOME |
| DRGVER | DRG or MS-DRG grouper version used on discharge date | OUTCOME |
| DRG_NoPOA | DRG in use on discharge date, calculated without POA | OUTCOME |
| HOSP_NIS| Hospital ID | Probably extraneous for now, though would be interesting to keep...| 
| KEY_NIS | Admission ID | No useful information |
| HOSPBRTH | HCUP indicator indicating in-hospital birth | Ignore neonates |
| LOS | Length of stay | OUTCOME |
| MDC | MDC (Major Diagnostic Category) in effect on discharge date	| OUTCOME |
| MDC24 | MDC (Major Diagnostic Category) version 24 | OUTCOME | 
| MDC_NoPOA | MDC (Major Diagnostic Category) in effect on discharge date, calc w/o POA | OUTCOME |
| NCHRONIC | Number of chronic conditions | No useful information |
| NDX | Number of ICD-9-CM diagnoses on this discharge | No useful information |
| NECODE | Number of ICD-9-cM EoCoI Codes on this Record | No useful information |
| NPR | Number of ICD-9-CM procedures on this discharge | No useful information |
| NEOMAT | Neonatal or maternal ICD-9-CM DX / PR | Ignore neonates |
| RACE | Race | For obvious reasons |
| TOTCHG | Total charges, cleaned (continuous) | OUTCOME |
| TRAN_OUT | Indicator of a transfer outside of the hospital | Categorical | [0, 1] |
| YEAR | Year of hospitalization | No useful information |
| APRDRG | All patient refined DRG, Categorical [0, 999] | Dependent feature class |
| APRDRG_Risk_Mortality | All Patient Refined DRG: Risk of Mortality Subclass, Categorical [0, 4] | Dependent feature class |
| APRDRG_Severity | All Patient Refined DRG: Severity of Illness Subclass, Categorical [0, 4] | Dependent feature class |

### Exclusion Criteria

First, let's identify how many values are missing from our feature set.

In [1]:
import numpy as np
import pandas as pd
import h5py
import re
import tables
import json

In [2]:
DATA_FOLDER = '../data/raw/'
MISS_VAL_FILL = -128
YEARS = ['2012', '2013', '2014']

In [3]:
OUTCOMES = ['DIED', 'DISPUNIFORM', 'DRG', 'LOS', 'TOTCHG', 'TRAN_OUT', 'APRDRG', 'APRDRG_SEVERITY', 'APRDRG_RISK_MORTALITY']

In [4]:
FEATURES = ['AGE', 
            'AMONTH', 
            'AWEEKEND', 
            'BODYSYSTEMn', 
            'CHRONBn',
            'CHRONn',
            'CM_AIDS',
            'DXn',
            'DXCCSn',
            'DXMCCSn'
            'E_CCS1',
            'E_MCCSn',
            'ECODEn',
            'ELECTIVE',
            'FEMALE',
            'HCUP_ED',
            'HOSP_DIVISION',
            'NIS_STRATUM',
            'ORPROC',
            'PAY1',
            'PCLASSn',
            'PL_NCHSn',
            'PRn',
            'PRCCSn',
            'PRDAYn',
            'PRMCCSn',
            'TRAN_IN',
            'TRAN_OUT',
            'ZIPINC_QRTL'
           ]

In [5]:
def find_feature(header):
    """
    """
    if header in FEATURES:
        return True
        
    elif header[:3] == 'CM_':
        return True
        
    else:
        prefix = re.split('\d', header)[0] # Prefix before #
        n_feature = prefix + 'n'
        
        if n_feature in FEATURES:
            return n_feature
        
        else:
            return False

In [175]:
def stats(series):
    num_missing = np.sum(series == -128)
#     unique, counts = np.unique(series, return_counts=True)
#     return {'missing': num_missing, 'unique': unique, 'counts': counts}
    return {'missing': num_missing}

In [5]:
data = h5py.File(DATA_FOLDER + 'NIS.h5', 'r')

In [6]:
with open(DATA_FOLDER + 'NIS_columnstats_2012_2014.json', 'r') as f:
    headers_found = json.load(f)

In [177]:
headers_found = {}

for year in data.keys():
    for header_idx, header_pre in enumerate(data[year]['{0}_headers'.format(year)]):
        header = header_pre.decode('utf-8').upper()
            
        if find_feature(header):
            
            if not (header in headers_found.keys()):
                headers_found[header] = {}

            series = data[year]['{0}_data'.format(year)][:, header_idx]
            headers_found[header][year] = stats(series)
            print("Year {0}, Header {1} Completed".format(year, header))

Year 2012, Header PRCCS1 Completed
Year 2012, Header PRCCS2 Completed
Year 2012, Header PRCCS3 Completed
Year 2012, Header PRCCS4 Completed
Year 2012, Header PRCCS5 Completed
Year 2012, Header PRCCS6 Completed
Year 2012, Header PRCCS7 Completed
Year 2012, Header PRCCS8 Completed
Year 2012, Header PRCCS9 Completed


KeyboardInterrupt: 

In [14]:
# Print missing values
import re
missing_num = 0
# missing_counts = pd.DataFrame()
for header, header_info in headers_found.items():
    if not re.search('\d', header):
        for year, year_info in header_info.items():
            missing_num += year_info['missing']
            if year_info['missing'] > 0:
                print("Field: {0} :: Year: {1} :: Number Missing: {2}".format(header, year, year_info['missing']))
                
print('\nTotal Missing ::: {0}'.format(missing_num))

Field: AMONTH :: Year: 2012 :: Number Missing: 9539
Field: AMONTH :: Year: 2013 :: Number Missing: 7535
Field: AMONTH :: Year: 2014 :: Number Missing: 7065
Field: AWEEKEND :: Year: 2012 :: Number Missing: 1946
Field: AWEEKEND :: Year: 2013 :: Number Missing: 188
Field: AWEEKEND :: Year: 2014 :: Number Missing: 13
Field: ELECTIVE :: Year: 2012 :: Number Missing: 25000
Field: ELECTIVE :: Year: 2013 :: Number Missing: 23980
Field: ELECTIVE :: Year: 2014 :: Number Missing: 23410
Field: FEMALE :: Year: 2012 :: Number Missing: 968
Field: FEMALE :: Year: 2013 :: Number Missing: 1445
Field: FEMALE :: Year: 2014 :: Number Missing: 1511
Field: TRAN_IN :: Year: 2012 :: Number Missing: 39432
Field: TRAN_IN :: Year: 2013 :: Number Missing: 31321
Field: TRAN_IN :: Year: 2014 :: Number Missing: 43478
Field: TRAN_OUT :: Year: 2012 :: Number Missing: 1814
Field: TRAN_OUT :: Year: 2013 :: Number Missing: 2532
Field: TRAN_OUT :: Year: 2014 :: Number Missing: 2964
Field: ZIPINC_QRTL :: Year: 2012 :: Numbe

Also really curious about these external causes of injury codes / cases...

In [15]:
for header, header_info in headers_found.items():
    if header[0:5] == 'ECODE' or header[0:2] == 'E_':
        for year, year_info in header_info.items():
            print("{0} :: {1} :: {2}".format(header, year, year_info['missing']))

E_MCCS1 :: 2012 :: 6150685
E_MCCS1 :: 2013 :: 5950562
E_MCCS1 :: 2014 :: 5917440
ECODE1 :: 2012 :: 6150685
ECODE1 :: 2013 :: 5950562
ECODE1 :: 2014 :: 5917440
ECODE2 :: 2012 :: 6725049
ECODE2 :: 2013 :: 6515150
ECODE2 :: 2014 :: 6498329
ECODE3 :: 2012 :: 7144155
ECODE3 :: 2013 :: 6924624
ECODE3 :: 2014 :: 6919133
ECODE4 :: 2012 :: 7241727
ECODE4 :: 2013 :: 7020036
ECODE4 :: 2014 :: 7022280


In [133]:
for header, header_info in headers_found.items():
    if header[0:5] == 'CHRON':
        for year, year_info in header_info.items():
            if year == '2012':
                print("{0} :: {1} :: {2}".format(header, year, year_info['missing']))

CHRON1 :: 2012 :: 6317
CHRON2 :: 2012 :: 289597
CHRON3 :: 2012 :: 905728
CHRON4 :: 2012 :: 1487181
CHRON5 :: 2012 :: 2030871
CHRON6 :: 2012 :: 2542930
CHRON7 :: 2012 :: 3024421
CHRON8 :: 2012 :: 3482539
CHRON9 :: 2012 :: 3926275
CHRON10 :: 2012 :: 4428846
CHRON11 :: 2012 :: 4786052
CHRON12 :: 2012 :: 5106297
CHRON13 :: 2012 :: 5397762
CHRON14 :: 2012 :: 5662559
CHRON15 :: 2012 :: 5905138
CHRON16 :: 2012 :: 6191875
CHRON17 :: 2012 :: 6429095
CHRON18 :: 2012 :: 6566660
CHRON19 :: 2012 :: 6798807
CHRON20 :: 2012 :: 6879838
CHRON21 :: 2012 :: 6952067
CHRON22 :: 2012 :: 7009797
CHRON23 :: 2012 :: 7058434
CHRON24 :: 2012 :: 7101335
CHRON25 :: 2012 :: 7145083
CHRONB1 :: 2012 :: 6317
CHRONB2 :: 2012 :: 289597
CHRONB3 :: 2012 :: 905728
CHRONB4 :: 2012 :: 1487181
CHRONB5 :: 2012 :: 2030871
CHRONB6 :: 2012 :: 2542930
CHRONB7 :: 2012 :: 3024421
CHRONB8 :: 2012 :: 3482539
CHRONB9 :: 2012 :: 3926275
CHRONB10 :: 2012 :: 4428846
CHRONB11 :: 2012 :: 4786052
CHRONB12 :: 2012 :: 5106297
CHRONB13 :: 2012 

In [16]:
for header, header_info in headers_found.items():
    if header[0:3] == 'AGE':
        for year, year_info in header_info.items():
            print("{0} :: {1} :: {3} :: {4}".format(header, year, year_info['missing'], year_info['unique'], year_info['counts']))

AGE :: 2012 :: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 128] :: [856053, 30925, 28566, 16205, 13915, 12716, 11613, 15990, 10656, 9494, 9769, 10431, 17138, 13220, 16068, 19211, 30586, 28567, 37607, 56747, 55118, 59400, 91503, 62944, 64016, 66178, 69121, 102583, 72491, 74945, 76180, 89025, 102568, 68137, 64166, 61242, 56093, 75804, 51109, 51371, 52276, 54739, 74685, 54003, 55227, 57227, 61198, 88550, 71132, 74563, 78173, 81669, 111322, 84936, 103694, 90200, 90047, 122676, 90955, 92286, 93081, 92727, 125341, 94517, 97991, 111183, 91082, 126932, 96710, 105215, 95310, 91671, 122244, 90617, 90480, 90048, 89132, 119383, 85990, 86382, 85882, 86541, 113984, 83430, 82608, 78538, 73402, 8

So, about a million cases that are related to 'external causes of injury'. May as well keep them. 

Besides these features, we should create some exclusion criteria:
1. Patients under the age of 18.
2. Patients with missing values for the following fields: Elective admission, Gender, Transfer In status, Zipcode income quartile,  Admission month, Admission weekend, and PLCHS (urban/rural classification scheme)

Other modifications to consider:
1. Combine PRDAYn with one-hot encoding for PRn.
2. One-hot encode all DX codes. Eek.
3. One-hot encode all PR codes.

In [41]:
for header, header_info in headers_found.items():
    if len(header_info.keys()) < 3:
        print(header)

CHRONB1
CHRONB2
CHRONB3
CHRONB4
CHRONB5
CHRONB6
CHRONB7
CHRONB8
CHRONB9
CHRONB10
CHRONB11
CHRONB12
CHRONB13
CHRONB14
CHRONB15
CHRONB16
CHRONB17
CHRONB18
CHRONB19
CHRONB20
CHRONB21
CHRONB22
CHRONB23
CHRONB24
CHRONB25
PL_NCHS2006
PL_NCHS
DX26
DX27
DX28
DX29
DX30
DXCCS26
DXCCS27
DXCCS28
DXCCS29
DXCCS30
BODYSYSTEM1
BODYSYSTEM2
BODYSYSTEM3
BODYSYSTEM4
BODYSYSTEM5
BODYSYSTEM6
BODYSYSTEM7
BODYSYSTEM8
BODYSYSTEM9
BODYSYSTEM10
BODYSYSTEM11
BODYSYSTEM12
BODYSYSTEM13
BODYSYSTEM14
BODYSYSTEM15
BODYSYSTEM16
BODYSYSTEM17
BODYSYSTEM18
BODYSYSTEM19
BODYSYSTEM20
BODYSYSTEM21
BODYSYSTEM22
BODYSYSTEM23
BODYSYSTEM24
BODYSYSTEM25
BODYSYSTEM26
BODYSYSTEM27
BODYSYSTEM28
BODYSYSTEM29
BODYSYSTEM30
CHRON26
CHRON27
CHRON28
CHRON29
CHRON30


(262,)

### "Final" Dataset Creation

In [ ]:
# Probably better to generate one-hot encodings of minibatches at train time rather than store it all in one big go now.

In [18]:
import tables

In [80]:
try: data_proc.close()
except: pass

DATA_PRUNED_FN = 'NIS_Pruned.h5'
data_proc = tables.open_file(DATA_FOLDER + DATA_PRUNED_FN, 'w')

data_proc.create_earray('/', 'key_nis', shape=(0, ), atom=tables.Int32Atom());

In [3]:
import json

with open(DATA_FOLDER + 'NIS_columnstats_2012_2014.json', 'r') as f:
    headers_found = json.load(f)

In [66]:
num_records = 0
for year, year_info in data.items():
    year_data = year_info['{0}_data'.format(year)]
    num_records += year_data.shape[0]

In [67]:
num_records

21438293

In [36]:
# keys_to_remove = np.empty((0, ), dtype='int32')
keys_to_remove = {}
remove_missing = (lambda series : np.where(series == -128))

# Define exclusion criteria
exclusion_criteria = {
    b'AGE' : (lambda series : np.where(series < 19)),
    b'ELECTIVE' : remove_missing,
    b'FEMALE' : remove_missing,
    b'TRAN_IN' : remove_missing,
    b'ZIPINC_QRTL' : remove_missing,
    b'HCUP_ED' : remove_missing,
#     b'AMONTH' : remove_missing,
#     b'AWEEKEND' : remove_missing,
#     b'PL_NCHS' : remove_missing,
#     b'PL_NCHS2006' : remove_missing,
}

excluded = 0

# Execute exclusion criteria
for year, year_info in data.items():
    year_headers = year_info['{0}_headers'.format(year)][:].tolist()
    year_data = year_info['{0}_data'.format(year)]
    
    keys_to_remove_yr = np.empty((0, ), dtype='int32')
    nis_keys = year_data[:, year_headers.index(b'KEY_NIS')]
    
    for exclusion_key, exclusion_func in exclusion_criteria.items():
        try:
            exclusion_idx = year_headers.index(exclusion_key)
        except:
            break
            
        exclusion_series = year_data[:, exclusion_idx]
        excluded_idxs = exclusion_func(exclusion_series)
        excluded_keys = nis_keys[excluded_idxs]
        
        keys_to_remove_yr = np.concatenate((keys_to_remove_yr, excluded_keys))
        
        print('Year :: {2}, Key :: {0}, Excluded Num :: {1}'.format(exclusion_key, len(excluded_keys), year))
        excluded += len(excluded_keys)
        
    keys_to_remove[year] = keys_to_remove_yr

Year :: 2012, Key :: b'AGE', Excluded Num :: 1188730
Year :: 2012, Key :: b'ELECTIVE', Excluded Num :: 25000
Year :: 2012, Key :: b'FEMALE', Excluded Num :: 968
Year :: 2012, Key :: b'TRAN_IN', Excluded Num :: 39432
Year :: 2012, Key :: b'ZIPINC_QRTL', Excluded Num :: 162514
Year :: 2012, Key :: b'HCUP_ED', Excluded Num :: 0
Year :: 2013, Key :: b'AGE', Excluded Num :: 1151208
Year :: 2013, Key :: b'ELECTIVE', Excluded Num :: 23980
Year :: 2013, Key :: b'FEMALE', Excluded Num :: 1445
Year :: 2013, Key :: b'TRAN_IN', Excluded Num :: 31321
Year :: 2013, Key :: b'ZIPINC_QRTL', Excluded Num :: 158891
Year :: 2013, Key :: b'HCUP_ED', Excluded Num :: 0
Year :: 2014, Key :: b'AGE', Excluded Num :: 1151123
Year :: 2014, Key :: b'ELECTIVE', Excluded Num :: 23410
Year :: 2014, Key :: b'FEMALE', Excluded Num :: 1511
Year :: 2014, Key :: b'TRAN_IN', Excluded Num :: 43478
Year :: 2014, Key :: b'ZIPINC_QRTL', Excluded Num :: 153861
Year :: 2014, Key :: b'HCUP_ED', Excluded Num :: 0


In [68]:
num_records - excluded

17281421

In [81]:
data_proc_headers = []
outcome_headers = []

for year, year_info in data.items():
    year_headers = year_info['{0}_headers'.format(year)][:].tolist()
#     year_header_idx_map = (-1 * np.ones(len(data_proc_headers), dtype='int16')).tolist() # Array that will store where year_headers lie in the data_proc_header list.


    for year_header_idx, year_header in enumerate(year_headers):

        # If we care about this header as an input feature...
        if year_header.decode('utf-8').upper() in headers_found.keys():
            
            if year_header == b'PL_NCHS2006':
                year_header_rectified = b'PL_NCHS'
                
            elif year_header[0:10] == b'BODYSYSTEM':
                n_idx = re.search('\d', year_header.decode('utf-8').upper()).start()
                n = int(year_header[n_idx:].decode('utf-8'))
                year_header_rectified = ('CHRONB{0:02d}'.format(n)).encode('ASCII')
            
            elif len(re.findall('\d', year_header.decode('utf-8'))) > 0:
                n_idx = re.search('\d', year_header.decode('utf-8').upper()).start()
                prefix = re.split('\d', year_header.decode('utf-8').upper())[0]
                n = int(year_header[n_idx:].decode('utf-8'))
                year_header_rectified = ('{0}{1:02d}'.format(prefix, n)).encode('ASCII')                
            
            else:
                year_header_rectified = year_header

            # If this key we need to include isn't already present...
            if not (year_header_rectified in data_proc_headers):
                data_proc_headers.append(year_header_rectified)
                
        elif year_header.decode('utf-8').upper() in OUTCOMES:
            if year_header not in outcome_headers:
                outcome_headers.append(year_header)


In [82]:
# Here, let's make sure all column names are consistent.

# data_proc_headers = []
year_idx_maps = []
excluded_idx_maps = []

for year, year_info in data.items():
    year_data = year_info['{0}_data'.format(year)]    
    year_keys = year_data[:, year_headers.index(b'KEY_NIS')]
    
    # Now, let's ensure that our column indices line up appropriately with the current state of the header list.
    year_headers = year_info['{0}_headers'.format(year)][:].tolist()
    year_header_idx_map = (-1 * np.ones(len(data_proc_headers), dtype='int16')).tolist() # Array that will store where year_headers lie in the data_proc_header list.
    excluded_idx_map = (-1 * np.ones(len(outcome_headers), dtype='int16')).tolist()
    
    for year_header_idx, year_header in enumerate(year_headers):

        # If we care about this header as an input feature...
        if year_header.decode('utf-8').upper() in headers_found.keys():
            
            if year_header == b'PL_NCHS2006':
                year_header_rectified = b'PL_NCHS'
                
            elif year_header[0:10] == b'BODYSYSTEM':
                n_idx = re.search('\d', year_header.decode('utf-8').upper()).start()
                n = int(year_header[n_idx:].decode('utf-8'))
                year_header_rectified = ('CHRONB{0:02d}'.format(n)).encode('ASCII')
            
            elif len(re.findall('\d', year_header.decode('utf-8'))) > 0:
                n_idx = re.search('\d', year_header.decode('utf-8').upper()).start()
                prefix = re.split('\d', year_header.decode('utf-8').upper())[0]
                n = int(year_header[n_idx:].decode('utf-8'))
                year_header_rectified = ('{0}{1:02d}'.format(prefix, n)).encode('ASCII')                
            
            else:
                year_header_rectified = year_header

            # If this key we need to include isn't already present...
            if not (year_header_rectified in data_proc_headers):
                year_header_idx_map.append(year_header_idx)
                    
            else:
                year_header_idx_map[data_proc_headers.index(year_header_rectified)] = year_header_idx
                
        elif year_header.decode('utf-8').upper() in OUTCOMES:
            if year_header not in outcome_headers:
                excluded_idx_map.append(year_header_idx)
            else:
                excluded_idx_map[outcome_headers.index(year_header)] = year_header_idx
                
                
    year_idx_maps.append(year_header_idx_map)
    excluded_idx_maps.append(excluded_idx_map)

for i, year_idx_map in enumerate(year_idx_maps):
    diff_in_len = len(data_proc_headers) - len(year_idx_map)
    if diff_in_len > 0:
        extension_len = [-1] * diff_in_len
        year_idx_maps[i].extend(extension_len)
        
sort_idx = np.argsort(data_proc_headers)
data_proc_headers = np.sort(data_proc_headers)

year_idx_maps_sorted = []
for i, year_idx_map in enumerate(year_idx_maps):
    year_idx_maps_sorted.append(np.array(year_idx_map)[sort_idx].tolist())
    
sort_idx = np.argsort(outcome_headers)
outcome_headers = np.sort(outcome_headers)

excluded_idx_maps_sorted = []
for i, excluded_idx_map in enumerate(excluded_idx_maps):
    excluded_idx_maps_sorted.append(np.array(excluded_idx_map)[sort_idx].tolist())

data_proc.create_array('/', 'headers', data_proc_headers)
data_proc.create_array('/', 'outcome_headers', outcome_headers)
data_proc.create_earray('/', 'dataset', shape=(0, len(data_proc_headers)), atom=tables.Int32Atom())
data_proc.create_earray('/', 'outcomes', shape=(0, len(outcome_headers)), atom=tables.Int32Atom())

/outcomes (EArray(0, 8)) ''
  atom := Int32Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (2048, 8)

In [83]:
# Now, let's remove excluded cases and combine all years in a single dataset! 

for idx, (year, year_info) in enumerate(data.items()):

    year_headers = year_info['{0}_headers'.format(year)][:].tolist()
    year_data = year_info['{0}_data'.format(year)]

    year_idx_map = year_idx_maps_sorted[idx]
    excluded_idx_map = excluded_idx_maps_sorted[idx]
    keys_to_remove_yr = keys_to_remove[year]
    
    count = 0
    chunk_size = 1000000
    while count < year_data.shape[0]:
        print("Starting {0} : {1}".format(count, count + chunk_size))

        if count > year_data.shape[0]:
            count = year_data.shape[0]

        chunk = year_data[count:count+chunk_size, :]
        
        # Do something
        year_keys = chunk[:, year_headers.index(b'KEY_NIS')]

        # Remove keys
        key_int_to_remove = np.in1d(year_keys, keys_to_remove_yr)
        #     idxs_to_remove = np.where(key_int_to_remove)

        key_int_to_keep = ~key_int_to_remove
        idxs_to_keep = np.where(key_int_to_keep)[0]
        
        year_data_pruned_nocols = np.take(chunk, idxs_to_keep, axis=0)
        year_data_pruned_nocols_ext = np.concatenate([year_data_pruned_nocols, (0 * np.ones((year_data_pruned_nocols.shape[0], 1))).astype('int32')], axis=1)
        year_data_pruned_cols = np.take(year_data_pruned_nocols_ext, year_idx_map, axis=1)
        
        outcome_year = np.take(year_data_pruned_nocols_ext, excluded_idx_map, axis=1)
        
        data_proc.root.dataset.append(year_data_pruned_cols)
        data_proc.root.key_nis.append(year_keys[idxs_to_keep])
        
        data_proc.root.outcomes.append(outcome_year)
        
          
        # Done
        count += chunk_size
        if count == year_data.shape[0]:
            break


Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000


In [101]:
data_proc.close()

In [85]:
data_proc = tables.open_file(DATA_FOLDER + DATA_PRUNED_FN, 'r')

In [100]:
data_proc.root.outcomes[0:1000, 3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    1,    0,    0,    1,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    1,    1,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    1,    0,    0,    0,   

### KEY_NIS is NOT a unique number across other years!

:(

#### Filter Out Admission Diagnoses vs. Pre-existing Comorbidities

In [1]:
import numpy as np
import pandas as pd
import h5py
import re
import tables
import json
from numba import jit

In [2]:
DATA_FOLDER = '../data/raw/'
MISS_VAL_FILL = -128
YEARS = ['2012', '2013', '2014']

In [17]:
data_proc = tables.open_file(DATA_FOLDER + 'NIS_Pruned.h5', 'r')
dataset = data_proc.root.dataset

In [55]:
try: data_proc.close()
except: pass

# data_proc = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_proto_emb_red.h5', 'w')
# dataset = data_proc.root.dataset

In [21]:
chron_idxs = []
dx_idxs = []
for header_idx, header in enumerate(data_proc.root.headers):
    if header[:5] == b'CHRON' and header[5:6] != b'B':
        chron_idxs.append(header_idx)
    if header[:2] == b'DX' and header[2:3] != b'C':
        dx_idxs.append(header_idx)
        
chron_idxs = np.array(chron_idxs, dtype='uint16')
dx_idxs = np.array(dx_idxs, dtype='uint16')

In [12]:
# @jit(nopython=True)
def remove_chrons_from_dxs(sample, chron_idxs, dx_idxs):
    sample_chrons = sample[:, chron_idxs].ravel()
    sample_dxs = sample[:, dx_idxs].ravel()
    mask = (sample_chrons == 1)
    sample_chrons = sample_dxs * mask
    mask.reshape(sample.shape[0], dx_idxs.shape[0])[:, 0] = 0
    mask = mask.ravel()
    sample_dxs[mask] = 0
    sample[:, dx_idxs] = sample_dxs.reshape(sample.shape[0], dx_idxs.shape[0])
    sample[:, chron_idxs] = sample_chrons.reshape(sample.shape[0], chron_idxs.shape[0])

In [67]:
%timeit remove_chrons_from_dxs(sample, chron_idxs, dx_idxs) # JIT result

208 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [106]:
%timeit remove_chrons_from_dxs(sample, chron_idxs, dx_idxs).reshape(sample_chrons.shape) # regular result

169 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
dataset = data_proc.root.dataset

count = 0
chunk_size = 1000000

while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))

    if count > dataset.shape[0]:
        count = dataset.shape[0]

    sample = dataset[count:count+chunk_size, :]
    remove_chrons_from_dxs(sample, chron_idxs, dx_idxs)
    dataset[count:count+chunk_size, :] = sample

    count += chunk_size
    if count == dataset.shape[0]:
        break

Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 8000000 : 9000000
Starting 9000000 : 10000000
Starting 10000000 : 11000000
Starting 11000000 : 12000000
Starting 12000000 : 13000000
Starting 13000000 : 14000000
Starting 14000000 : 15000000
Starting 15000000 : 16000000
Starting 16000000 : 17000000
Starting 17000000 : 18000000


#### Set Absent Procedures / Diagnoses to 0 for Embedding

In [34]:
embedding_idxs

array([  3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
        16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 152,
       153, 154, 155], dtype=uint16)

In [33]:
embedding_idxs = []
for header_idx, header in enumerate(data_proc.root.headers):
    if header[:6] == b'PR' or (header[:2] == b'DX' and header[2:3] != b'C') or (header[:5] == b'ECODE') or (header[:5] == b'CHRON'):
        embedding_idxs.append(header_idx)
        
embedding_idxs = np.array(embedding_idxs, dtype='uint16')

In [35]:
@jit(nopython=True)
def remove_negative_vals(sample, embedding_idxs):
    embeds = sample[:, embedding_idxs].ravel()
    mask = (embeds == -128)
    embeds[mask] = 0
    sample[:, embedding_idxs] = embeds.reshape(sample.shape[0], embedding_idxs.shape[0])

In [37]:
count = 0
chunk_size = 1000000
while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))
    
    # iteration
    if count > dataset.shape[0]:
        count = dataset.shape[0]
    
    # do something
    chunk = dataset[count:count+chunk_size, :]
    remove_negative_vals(chunk, embedding_idxs)
    dataset[count:count+chunk_size, :] = chunk

    # finalize iteration
    count += chunk_size
    if count == dataset.shape[0]:
        break


Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 8000000 : 9000000
Starting 9000000 : 10000000
Starting 10000000 : 11000000
Starting 11000000 : 12000000
Starting 12000000 : 13000000
Starting 13000000 : 14000000
Starting 14000000 : 15000000
Starting 15000000 : 16000000
Starting 16000000 : 17000000
Starting 17000000 : 18000000


#### Remove Extra Columns for Our First Autoencoder Prototype

In [7]:
# headers_to_keep = [b'DXn', b'PRn', b'ECODEn', b'ELECTIVE', b'FEMALE', b'AGE', b'HCUP_ED', b'ZIPINC_QRTL', b'CHRONBn', b'CHRONn', b'TRAN_IN', 'CHRON']
headers_to_keep = [b'DXn', b'DXCCSn', b'PRn', b'ECODEn', b'ELECTIVE', b'FEMALE', b'AGE', b'HCUP_ED', b'ZIPINC_QRTL', b'CHRONBn', b'CHRONn', b'TRAN_IN', 'CHRON']

In [8]:
def find_feature(header, keepers):
    """
    """
        
    if header in keepers:
        return True
                
    else:
        header = header.decode('utf-8')
        prefix = re.split('\d', header)[0] # Prefix before #
        n_feature = (prefix + 'n').encode('ASCII')
        
        if n_feature in keepers:
            return n_feature
        
        else:
            return False

In [9]:
data_proc = tables.open_file(DATA_FOLDER + 'NIS_Pruned.h5', 'r')

In [10]:
headers_to_keep_all = []
for header in data_proc.root.headers:
    if find_feature(header, headers_to_keep):
        headers_to_keep_all.append(header)

In [11]:
headers_to_keep_all

[b'AGE',
 b'CHRON01',
 b'CHRON02',
 b'CHRON03',
 b'CHRON04',
 b'CHRON05',
 b'CHRON06',
 b'CHRON07',
 b'CHRON08',
 b'CHRON09',
 b'CHRON10',
 b'CHRON11',
 b'CHRON12',
 b'CHRON13',
 b'CHRON14',
 b'CHRON15',
 b'CHRON16',
 b'CHRON17',
 b'CHRON18',
 b'CHRON19',
 b'CHRON20',
 b'CHRON21',
 b'CHRON22',
 b'CHRON23',
 b'CHRON24',
 b'CHRON25',
 b'CHRON26',
 b'CHRON27',
 b'CHRON28',
 b'CHRON29',
 b'CHRON30',
 b'CHRONB01',
 b'CHRONB02',
 b'CHRONB03',
 b'CHRONB04',
 b'CHRONB05',
 b'CHRONB06',
 b'CHRONB07',
 b'CHRONB08',
 b'CHRONB09',
 b'CHRONB10',
 b'CHRONB11',
 b'CHRONB12',
 b'CHRONB13',
 b'CHRONB14',
 b'CHRONB15',
 b'CHRONB16',
 b'CHRONB17',
 b'CHRONB18',
 b'CHRONB19',
 b'CHRONB20',
 b'CHRONB21',
 b'CHRONB22',
 b'CHRONB23',
 b'CHRONB24',
 b'CHRONB25',
 b'CHRONB26',
 b'CHRONB27',
 b'CHRONB28',
 b'CHRONB29',
 b'CHRONB30',
 b'DX01',
 b'DX02',
 b'DX03',
 b'DX04',
 b'DX05',
 b'DX06',
 b'DX07',
 b'DX08',
 b'DX09',
 b'DX10',
 b'DX11',
 b'DX12',
 b'DX13',
 b'DX14',
 b'DX15',
 b'DX16',
 b'DX17',
 b'DX18',
 

In [12]:
prototype_data = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_v2_ccs.h5', 'w')

In [13]:
prototype_data.create_carray('/', 'dataset', atom=tables.Int32Atom(), shape=(data_proc.root.dataset.shape[0], len(headers_to_keep_all)))

/dataset (CArray(17369821, 145)) ''
  atom := Int32Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (903, 145)

In [18]:
for i, header in enumerate(headers_to_keep_all):
    idx = data_proc.root.headers[:].tolist().index(header)
    print(i, header, idx)
    prototype_data.root.dataset[:, i] = dataset[:, idx]

0 b'AGE' 0
1 b'CHRON01' 3
2 b'CHRON02' 4
3 b'CHRON03' 5
4 b'CHRON04' 6
5 b'CHRON05' 7
6 b'CHRON06' 8
7 b'CHRON07' 9
8 b'CHRON08' 10
9 b'CHRON09' 11
10 b'CHRON10' 12
11 b'CHRON11' 13
12 b'CHRON12' 14
13 b'CHRON13' 15
14 b'CHRON14' 16
15 b'CHRON15' 17
16 b'CHRON16' 18
17 b'CHRON17' 19
18 b'CHRON18' 20
19 b'CHRON19' 21
20 b'CHRON20' 22
21 b'CHRON21' 23
22 b'CHRON22' 24
23 b'CHRON23' 25
24 b'CHRON24' 26
25 b'CHRON25' 27
26 b'CHRON26' 28
27 b'CHRON27' 29
28 b'CHRON28' 30
29 b'CHRON29' 31
30 b'CHRON30' 32
31 b'CHRONB01' 33
32 b'CHRONB02' 34
33 b'CHRONB03' 35
34 b'CHRONB04' 36
35 b'CHRONB05' 37
36 b'CHRONB06' 38
37 b'CHRONB07' 39
38 b'CHRONB08' 40
39 b'CHRONB09' 41
40 b'CHRONB10' 42
41 b'CHRONB11' 43
42 b'CHRONB12' 44
43 b'CHRONB13' 45
44 b'CHRONB14' 46
45 b'CHRONB15' 47
46 b'CHRONB16' 48
47 b'CHRONB17' 49
48 b'CHRONB18' 50
49 b'CHRONB19' 51
50 b'CHRONB20' 52
51 b'CHRONB21' 53
52 b'CHRONB22' 54
53 b'CHRONB23' 55
54 b'CHRONB24' 56
55 b'CHRONB25' 57
56 b'CHRONB26' 58
57 b'CHRONB27' 59
58 b'CHRO

In [47]:
prototype_data.close()

### Split into Training and Test Sets

In [19]:
splits = {'TRAIN': {'split': 0.6}, 
          'VAL': {'split': 0.2},
          'TEST': {'split': 0.2}
         }

In [20]:
split_inds = np.arange(0, dataset.shape[0], 1)

In [21]:
np.random.seed(seed=0)

for shuffle_x in range(10):
    np.random.shuffle(split_inds)

In [22]:
lower_bound_ind = 0
for subset, subset_info in splits.items():
    upper_bound_ind = int(np.round(lower_bound_ind + subset_info['split'] * dataset.shape[0]))
    splits[subset]['inds'] = split_inds[lower_bound_ind:upper_bound_ind].tolist()
    lower_bound_ind = upper_bound_ind

In [23]:
with open(DATA_FOLDER + 'NIS_2012_2014_prototype_TVTsplit_v2_ccs.json', 'w') as f:
    json.dump(splits, f)

In [58]:
prototype_data.close()
prototype_data = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_v2.h5', 'a')

In [24]:
for subset, subset_info in splits.items():
    prototype_data.create_array(prototype_data.root, subset, np.array(subset_info['inds'], dtype='int32'))

In [25]:
prototype_data.create_array(prototype_data.root, 'headers', headers_to_keep_all)

/headers (Array(145,)) ''
  atom := StringAtom(itemsize=14, shape=(), dflt=b'')
  maindim := 0
  flavor := 'python'
  byteorder := 'irrelevant'
  chunkshape := None

In [26]:
prototype_data.close()

#### Convert to float

In [66]:
prototype_data_float

<closed File>

In [77]:
prototype_data_float.close()

In [27]:
prototype_data = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_v2_ccs.h5', 'r')
prototype_data_float = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_float_v2_ccs.h5', 'a')

prototype_data_float.create_array('/', 'dataset', shape=prototype_data.root.dataset.shape, atom=tables.Float32Atom())
prototype_data_float.create_array('/', 'TRAIN', prototype_data.root.TRAIN[:])
prototype_data_float.create_array('/', 'VAL', prototype_data.root.VAL[:])
prototype_data_float.create_array('/', 'TEST', prototype_data.root.TEST[:])
prototype_data_float.create_array('/', 'headers', prototype_data.root.headers[:])

/headers (Array(145,)) ''
  atom := StringAtom(itemsize=11, shape=(), dflt=b'')
  maindim := 0
  flavor := 'python'
  byteorder := 'irrelevant'
  chunkshape := None

In [28]:
count = 0
chunk_size = 1000000
dataset = prototype_data.root.dataset

while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))
    
    # iteration
    if count > dataset.shape[0]:
        count = dataset.shape[0]
    
    # do something
    chunk = dataset[count:count+chunk_size, :].astype('float32')
    prototype_data_float.root.dataset[count:count+chunk_size, :] = chunk

    # finalize iteration
    count += chunk_size
    if count == dataset.shape[0]:
        break

Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 8000000 : 9000000
Starting 9000000 : 10000000
Starting 10000000 : 11000000
Starting 11000000 : 12000000
Starting 12000000 : 13000000
Starting 13000000 : 14000000
Starting 14000000 : 15000000
Starting 15000000 : 16000000
Starting 16000000 : 17000000
Starting 17000000 : 18000000


In [29]:
prototype_data_float.close()
prototype_data.close()

### Zero Mean, Unit Variance Age

In [30]:
import tables
import numpy as np
import json
from numba import jit

DATA_FOLDER = '../data/raw/'

In [31]:
try: prototype_data.close()
except: pass

prototype_data_float = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_float_v2_ccs.h5', 'a')
# prototype_data_safety = tables.open_file(DATA_FOLDER + 'safety/NIS_2012_2014_prototype.h5', 'r')

In [32]:
age = prototype_data_float.root.dataset[:, 0]

In [33]:
@jit(nopython=True)
def normalize(array):
    mean, var = np.mean(array), np.var(array)
    zero_mean = array - mean
    unit_var = zero_mean / var
    return unit_var, mean, var

# warmup
normalize(np.random.rand(10000).astype(np.float));

In [35]:
prototype_data_float.root.dataset[:, 0] = normalize

TypeError: objects of type ``CPUDispatcher`` are not supported in this context, sorry; supported objects are: NumPy array, record or scalar; homogeneous list or tuple, integer, float, complex or bytes

In [36]:
prototype_data_float.close()

#### Bucket Age Into Decades

In [37]:
prototype_data_embred = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_float_v2_ccs.h5', 'a')

In [38]:
age_normalized = prototype_data_embred.root.dataset[:1000, 0]

In [39]:
age_normalized = (age_normalized * var) + mean

NameError: name 'var' is not defined

In [40]:
count = 0
chunk_size = 1000000
dataset = prototype_data_embred.root.dataset

while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))
    
    # iteration
    if count > dataset.shape[0]:
        count = dataset.shape[0]
    
    # do something
    chunk_age = dataset[count:count+chunk_size, 0]
    
#     chunk_age *= var
#     chunk_age += mean
    
    decade = np.floor(chunk_age / 10)
    
    prototype_data_embred.root.dataset[count:count+chunk_size, 0] = decade

    # finalize iteration
    count += chunk_size
    if count == dataset.shape[0]:
        break

Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 8000000 : 9000000
Starting 9000000 : 10000000
Starting 10000000 : 11000000
Starting 11000000 : 12000000
Starting 12000000 : 13000000
Starting 13000000 : 14000000
Starting 14000000 : 15000000
Starting 15000000 : 16000000
Starting 16000000 : 17000000
Starting 17000000 : 18000000


In [41]:
unique_decades = np.unique(prototype_data_embred.root.dataset[:, 0])

In [42]:
len(unique_decades)

10

In [43]:
prototype_data_embred.close()

### One Hot Encoding Num Classes

In [44]:
import os
os.listdir(DATA_FOLDER)

['NIS_cc_stroke_sgcs_psminput.csv',
 'NIS_stroke_annoyindex_controls_100trees.ann',
 'NIS_cc_stroke_psminput.csv',
 'NIS_2014_Total.h5',
 'safety',
 'NIS_stroke_casecontrol_100trees.ann',
 'archive',
 'NIS_columnstats_2012_2014.json',
 'NIS_cc_stroke_tableonedata.csv',
 'NIS_2012_2014_prototype_float_v2.h5',
 'psm_res',
 'NIS_2012_Total.h5',
 'NIS_2012_2014_proto_emb_v2.h5',
 'NIS_stroke_casecontrol_orig.h5',
 'Figure5_NumPVals.png',
 'NIS_stroke_casecontrol.h5',
 'NIS_controlindices.npy',
 'NIS_2012_2014_prototype_float_v2_ccs.h5',
 'NIS_2012_2014_proto_emb_v2_latentspace.h5',
 'NIS_2012_2014_prototype_v2_ccs.h5',
 'Figure4_PValueVisualization.png',
 'NIS_2012_2014_prototype_TVTsplit_v2_ccs.json',
 'NIS.h5',
 'NIS_2012_2014_proto_emb_v2_anntree.h5',
 'NIS_stroke_casecontrol_mod.h5',
 'NIS_stroke_annoyindex_cases_100trees.ann',
 '.DS_Store',
 'NIS_2013_Total.h5',
 'NIS_Pruned.h5']

In [45]:
import json
with open(DATA_FOLDER + 'NIS_columnstats_2012_2014.json', 'r') as f:
    headers_found = json.load(f)

In [46]:
# Find the unique values
unique_dx_codes_year = []
for header, header_info in headers_found.items():
    if header[:2] == 'DX' and header[2] in [str(i) for i in range(1, 10)]:
        print(header)
        for year_info in header_info.values():
            unique_els = year_info['unique']
            unique_dx_codes_year.extend(unique_els)

unique_dx_codes = np.unique(unique_dx_codes_year)
unique_dx_codes[unique_dx_codes == -128] = 0
dx_mapping = {code : mapping for code, mapping in zip(unique_dx_codes, np.arange(0, len(unique_dx_codes)))}
print("Total unique DX codes: {0}".format(len(unique_dx_codes)))

DX1
DX2
DX3
DX4
DX5
DX6
DX7
DX8
DX9
DX10
DX11
DX12
DX13
DX14
DX15
DX16
DX17
DX18
DX19
DX20
DX21
DX22
DX23
DX24
DX25
DX26
DX27
DX28
DX29
DX30
Total unique DX codes: 12583


In [47]:
# Find the unique values
unique_pr_codes_year = []
for header, header_info in headers_found.items():
    if header[:2] == 'PR' and header[2] in [str(i) for i in range(1, 10)]:
        print(header)
        for year_info in header_info.values():
            unique_els = year_info['unique']
            unique_pr_codes_year.extend(unique_els)

unique_pr_codes = np.unique(unique_pr_codes_year)
unique_pr_codes[unique_pr_codes == -128] = 0
pr_mapping = {code : mapping for code, mapping in zip(unique_pr_codes, np.arange(0, len(unique_pr_codes)))}
print("Total unique PR codes: {0}".format(len(unique_pr_codes)))

PR1
PR2
PR3
PR4
PR5
PR6
PR7
PR8
PR9
PR10
PR11
PR12
PR13
PR14
PR15
Total unique PR codes: 4445


In [48]:
# Find the unique values
unique_ei_codes_year = []
for header, header_info in headers_found.items():
    if header[:5] == 'ECODE' and header[5] in [str(i) for i in range(1, 10)]:
        print(header)
        for year_info in header_info.values():
            unique_els = year_info['unique']
            unique_ei_codes_year.extend(unique_els)

unique_ei_codes = np.unique(unique_ei_codes_year)
unique_ei_codes[unique_ei_codes == -128] = 0
ei_mapping = {code : mapping for code, mapping in zip(unique_ei_codes, np.arange(0, len(unique_ei_codes)))}
print("Total unique ECODES codes: {0}".format(len(unique_ei_codes)))

ECODE1
ECODE2
ECODE3
ECODE4
Total unique ECODES codes: 1186


Now, let's map each of these codes onto unique indices and save this mapping. 

In [49]:
# Find the unique values
unique_ccs_codes_year = []
for header, header_info in headers_found.items():
    if header[:5] == 'DXCCS' and header[5] in [str(i) for i in range(1, 10)]:
        print(header)
        for year_info in header_info.values():
            unique_els = year_info['unique']
            unique_ccs_codes_year.extend(unique_els)

unique_ccs_codes = np.unique(unique_ccs_codes_year)
unique_ccs_codes[unique_ccs_codes == -128] = 0
ccs_mapping = {code : mapping for code, mapping in zip(unique_ccs_codes, np.arange(0, len(unique_ccs_codes)))}
print("Total unique DXCCS codes: {0}".format(len(unique_ccs_codes)))

DXCCS1
DXCCS2
DXCCS3
DXCCS4
DXCCS5
DXCCS6
DXCCS7
DXCCS8
DXCCS9
DXCCS10
DXCCS11
DXCCS12
DXCCS13
DXCCS14
DXCCS15
DXCCS16
DXCCS17
DXCCS18
DXCCS19
DXCCS20
DXCCS21
DXCCS22
DXCCS23
DXCCS24
DXCCS25
DXCCS26
DXCCS27
DXCCS28
DXCCS29
DXCCS30
Total unique DXCCS codes: 263


In [50]:
try: prototype_data.close()
except: pass

prototype_data = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_float_v2_ccs.h5', 'a')
dataset = prototype_data.root.dataset

In [51]:
prototype_data_embmap = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_proto_emb_v2_ccs.h5', 'a')

In [53]:
prototype_data_embmap.create_array('/', 'dataset', shape=dataset.shape, atom=tables.Float32Atom())
prototype_data_embmap.create_array('/', 'TRAIN', prototype_data.root.TRAIN[:])
prototype_data_embmap.create_array('/', 'VAL', prototype_data.root.VAL[:])
prototype_data_embmap.create_array('/', 'TEST', prototype_data.root.TEST[:])
prototype_data_embmap.create_array('/', 'headers', prototype_data.root.headers[:]);

In [54]:
dx_map = np.vstack((np.array(list(dx_mapping.keys())), np.array(list(dx_mapping.values())))).T
pr_map = np.vstack((np.array(list(pr_mapping.keys())), np.array(list(pr_mapping.values())))).T
ei_map = np.vstack((np.array(list(ei_mapping.keys())), np.array(list(ei_mapping.values())))).T
ccs_map = np.vstack((np.array(list(ccs_mapping.keys())), np.array(list(ccs_mapping.values())))).T

prototype_data_embmap.create_group('/', 'mapping')
prototype_data_embmap.create_array('/mapping', 'CHRONn', dx_map)
prototype_data_embmap.create_array('/mapping', 'DXn', dx_map)
prototype_data_embmap.create_array('/mapping', 'PRn', pr_map)
prototype_data_embmap.create_array('/mapping', 'ECODEn', ei_map)
prototype_data_embmap.create_array('/mapping', 'DXCCSn', ccs_map)

/mapping/DXCCSn (Array(263, 2)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [55]:
def find_feature(pattern, headers):
    """
    """
    idxs = []
    for idx, header in enumerate(headers):
        header = header.decode('utf-8')
        lp = len(pattern)
        if header[:lp] == pattern and header[lp] in [str(i) for i in range(0, 10)]:
            idxs.append(idx)
    return idxs

In [56]:
chron_idxs = find_feature('CHRON', prototype_data_embmap.root.headers[:])
dx_idxs = find_feature('DX', prototype_data_embmap.root.headers[:])
pr_idxs = find_feature('PR', prototype_data_embmap.root.headers[:])
ei_idxs = find_feature('ECODE', prototype_data_embmap.root.headers[:])
ccs_idxs = find_feature('DXCCS', prototype_data_embmap.root.headers[:])

In [57]:
def map_code_to_map(array, idxs, mapping):
    for idx in idxs:
        array[:, idx] = mapping[array[:, idx]]

In [58]:
count = 0
chunk_size = 1000000
dataset = prototype_data.root.dataset

while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))
    
    # iteration
    if count > dataset.shape[0]:
        count = dataset.shape[0]
    
    # do something
    chunk = dataset[count:count+chunk_size, :].astype('float32')
    prototype_data_embmap.root.dataset[count:count+chunk_size, :] = chunk

    # finalize iteration
    count += chunk_size
    if count == dataset.shape[0]:
        break

Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 8000000 : 9000000
Starting 9000000 : 10000000
Starting 10000000 : 11000000
Starting 11000000 : 12000000
Starting 12000000 : 13000000
Starting 13000000 : 14000000
Starting 14000000 : 15000000
Starting 15000000 : 16000000
Starting 16000000 : 17000000
Starting 17000000 : 18000000


In [59]:
kdx = np.array(list(dx_mapping.keys()))
vdx = np.array(list(dx_mapping.values()))

mapping_dx = np.zeros(kdx.max()+1, dtype=vdx.dtype) #k,v from approach #1
mapping_dx[kdx] = vdx

kpr = np.array(list(pr_mapping.keys()))
vpr = np.array(list(pr_mapping.values()))

mapping_pr = np.zeros(kpr.max()+1, dtype=vpr.dtype) #k,v from approach #1
mapping_pr[kpr] = vpr

kei = np.array(list(ei_mapping.keys()))
vei = np.array(list(ei_mapping.values()))

mapping_ei = np.zeros(kei.max()+1, dtype=vei.dtype) #k,v from approach #1
mapping_ei[kei] = vei

kci = np.array(list(ccs_mapping.keys()))
vci = np.array(list(ccs_mapping.values()))

mapping_ccs = np.zeros(kci.max()+1, dtype=vci.dtype) #k,v from approach #1
mapping_ccs[kci] = vci



In [60]:
count = 0
chunk_size = 1000000
dataset = prototype_data.root.dataset


# out = mapping_ar[input_array]

while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))
    
    # iteration
    if count > dataset.shape[0]:
        count = dataset.shape[0]
    
    # do something
    chunk = dataset[count:count+chunk_size, :]
    
    chunk[:, chron_idxs] = mapping_dx[chunk[:, chron_idxs].ravel().astype('int64')].reshape(-1, len(chron_idxs))
    chunk[:, dx_idxs] = mapping_dx[chunk[:, dx_idxs].ravel().astype('int64')].reshape(-1, len(dx_idxs))
    chunk[:, pr_idxs] = mapping_pr[chunk[:, pr_idxs].ravel().astype('int64')].reshape(-1, len(pr_idxs))
    chunk[:, ei_idxs] = mapping_ei[chunk[:, ei_idxs].ravel().astype('int64')].reshape(-1, len(ei_idxs))
    chunk[:, ccs_idxs] = mapping_ccs[chunk[:, ccs_idxs].ravel().astype('int64')].reshape(-1, len(ccs_idxs))
    
    prototype_data_embmap.root.dataset[count:count+chunk_size, :] = chunk

    # finalize iteration
    count += chunk_size
    if count == dataset.shape[0]:
        break

Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 8000000 : 9000000
Starting 9000000 : 10000000
Starting 10000000 : 11000000
Starting 11000000 : 12000000
Starting 12000000 : 13000000
Starting 13000000 : 14000000
Starting 14000000 : 15000000
Starting 15000000 : 16000000
Starting 16000000 : 17000000
Starting 17000000 : 18000000


In [62]:
prototype_data_embmap.root.dataset[:, ccs_idxs].ravel().max()

262.0

In [63]:
prototype_data_embmap.close()
prototype_data.close()

#### Reducing Dimension Space for DX Features

In [185]:
prototype_data = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_prototype_float_v2.h5', 'a')
prototype_data_embmap = tables.open_file(DATA_FOLDER + 'NIS_2012_2014_proto_emb_red_v2.h5', 'a')

prototype_data_embmap.create_array('/', 'dataset', shape=dataset.shape, atom=tables.Float32Atom())
prototype_data_embmap.create_array('/', 'TRAIN', prototype_data.root.TRAIN[:])
prototype_data_embmap.create_array('/', 'VAL', prototype_data.root.VAL[:])
prototype_data_embmap.create_array('/', 'TEST', prototype_data.root.TEST[:])
prototype_data_embmap.create_array('/', 'headers', prototype_data.root.headers[:])

prototype_data_embmap.create_group('/', 'mapping')
prototype_data_embmap.create_array('/mapping', 'CHRONn', dx_map)
prototype_data_embmap.create_array('/mapping', 'DXn', dx_map)
prototype_data_embmap.create_array('/mapping', 'PRn', pr_map)
prototype_data_embmap.create_array('/mapping', 'ECODEn', ei_map)

NodeError: group ``/`` already has a child node named ``dataset``

In [52]:
from numba import jit

In [149]:
MISS_VAL_FILL = 0

@jit(nopython=True)
def reduce_icd9_complexity(array, depth=1):
    """Map numerical rep of ICD9 code to ICD9 code"""
#     cs = str(int(code))
#     len_code = len(cs)
    
#     if code >= 200000:
#         # external injury code (E000.0-E999.9) => (E000 - E990)
#         b = 200000
#         scale = 10 ** (5 - len_code)
        
#     elif code >= 100000:
#         # supplementary code (V01.00-V99.99) => (V01 - V99)
#         b = 100000
#         scale = 10 ** (4 - len_code)
        
#     elif code < 100000:
#         # regular ICD9 code (000.00-999.99) => (000 - 999)
#         b = 0
#         scale = 10 ** (5 - len_code)
        
#     else:
#         raise ValueError("Something went wrong with {0}".format(code))
        
#     original_code = (str(code) - b) / scale
#     print(original_code)
#     new_code = str(original_code)[:-depth]
    return np.floor(array / 10**(depth))

First, let's reduce the numerical ICD9 codes to the desired depth.

In [186]:
count = 0
chunk_size = 1000000
dataset = prototype_data.root.dataset

while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))
    
    # iteration
    if count > dataset.shape[0]:
        count = dataset.shape[0]
    
    # do something
    chunk = dataset[count:count+chunk_size, :]
    
    chunk[:, chron_idxs] = reduce_icd9_complexity(chunk[:, chron_idxs].ravel()).reshape(-1, len(chron_idxs))
    chunk[:, dx_idxs] = reduce_icd9_complexity(chunk[:, dx_idxs].ravel()).reshape(-1, len(dx_idxs))
    chunk[:, pr_idxs] = reduce_icd9_complexity(chunk[:, pr_idxs].ravel()).reshape(-1, len(pr_idxs))
    chunk[:, ei_idxs] = reduce_icd9_complexity(chunk[:, ei_idxs].ravel()).reshape(-1, len(ei_idxs))
    
    prototype_data_embmap.root.dataset[count:count+chunk_size, :] = chunk

    # finalize iteration
    count += chunk_size
    if count == dataset.shape[0]:
        break

Starting 0 : 1000000
Starting 1000000 : 2000000
Starting 2000000 : 3000000
Starting 3000000 : 4000000
Starting 4000000 : 5000000
Starting 5000000 : 6000000
Starting 6000000 : 7000000
Starting 7000000 : 8000000
Starting 8000000 : 9000000
Starting 9000000 : 10000000
Starting 10000000 : 11000000
Starting 11000000 : 12000000
Starting 12000000 : 13000000
Starting 13000000 : 14000000
Starting 14000000 : 15000000
Starting 15000000 : 16000000
Starting 16000000 : 17000000
Starting 17000000 : 18000000


In [187]:
prototype_data_embmap.root.dataset[:, dx_idxs].ravel().max(), prototype_data_embmap.root.dataset[:, chron_idxs].ravel().max()

(10919.0, 10881.0)

In [188]:
def map_chunk_codes_reduced_by_depth(chunk, patterns, depth):
    dx_maps = {}
    # Find unique codes first
    for pattern in patterns:
        
        if pattern == 'CHRON':
            search_pattern = 'DX'
        else:
            search_pattern = pattern
        
        lp = len(search_pattern)
        unique_dx_codes_year = []
        for header, header_info in headers_found.items():
            if header[:lp] == search_pattern and header[lp] in [str(i) for i in range(1, 10)]:
    #             print(header)
                for year_info in header_info.values():
                    unique_els = year_info['unique']
                    unique_dx_codes_year.extend(unique_els)
        
                    
        unique_dx_codes = np.unique(unique_dx_codes_year)
        unique_dx_codes[unique_dx_codes == -128] = 0

        unique_dx_codes = np.unique(reduce_icd9_complexity(unique_dx_codes, depth)).astype('int64')

        dx_mapping = {code : mapping for code, mapping in zip(unique_dx_codes, np.arange(0, len(unique_dx_codes)))}
        print("Total unique {1} codes: {0}".format(len(unique_dx_codes), pattern))

        dx_idxs = find_feature(pattern, prototype_data_embmap.root.headers[:])

        dx_maps[pattern] = np.vstack((np.array(list(dx_mapping.keys())), np.array(list(dx_mapping.values())))).T

        kdx = np.array(list(dx_mapping.keys()))
        vdx = np.array(list(dx_mapping.values()))

        mapping_dx = np.zeros(kdx.max()+1, dtype=vdx.dtype) #k,v from approach #1
        mapping_dx[kdx] = vdx
        
        chunk[:, dx_idxs] = mapping_dx[chunk[:, dx_idxs].ravel().astype('int64')].reshape(-1, len(dx_idxs))
    
    return chunk, dx_maps

In [192]:
count = 0
chunk_size = 1000000
dataset = prototype_data_embmap.root.dataset

while count < dataset.shape[0]:

    print("Starting {0} : {1}".format(count, count + chunk_size))
    
    # iteration
    if count > dataset.shape[0]:
        count = dataset.shape[0]
    
    # do something
    chunk = dataset[count:count+chunk_size, :]
    
    chunk, maps = map_chunk_codes_reduced_by_depth(chunk, ['CHRON', 'DX', 'PR', 'ECODE'], 1)
    
    if count == 0:
        for k, kmap in maps.items():
            setattr(prototype_data_embmap.root.mapping, ('{0}n'.format(k)), kmap)
    
    prototype_data_embmap.root.dataset[count:count+chunk_size, :] = chunk

    # finalize iteration
    count += chunk_size
    if count == dataset.shape[0]:
        break

Starting 0 : 1000000
Total unique CHRON codes: 6202
Total unique DX codes: 6202
Total unique PR codes: 3722
Total unique ECODE codes: 182
Starting 1000000 : 2000000
Total unique CHRON codes: 6202
Total unique DX codes: 6202
Total unique PR codes: 3722
Total unique ECODE codes: 182
Starting 2000000 : 3000000
Total unique CHRON codes: 6202
Total unique DX codes: 6202
Total unique PR codes: 3722
Total unique ECODE codes: 182
Starting 3000000 : 4000000
Total unique CHRON codes: 6202
Total unique DX codes: 6202
Total unique PR codes: 3722
Total unique ECODE codes: 182
Starting 4000000 : 5000000
Total unique CHRON codes: 6202
Total unique DX codes: 6202
Total unique PR codes: 3722
Total unique ECODE codes: 182
Starting 5000000 : 6000000
Total unique CHRON codes: 6202
Total unique DX codes: 6202
Total unique PR codes: 3722
Total unique ECODE codes: 182
Starting 6000000 : 7000000
Total unique CHRON codes: 6202
Total unique DX codes: 6202
Total unique PR codes: 3722
Total unique ECODE codes: 18

In [193]:
prototype_data_embmap.root.dataset[:, dx_idxs].ravel().max(), prototype_data_embmap.root.dataset[:, chron_idxs].ravel().max()

(6201.0, 6189.0)

In [195]:
prototype_data.close()
prototype_data_embmap.close()